# Machine Learning with Big Data / Assignment - III - b / Serap Aydogdu

2. Please find an unique dataset (from your work data or manipulated dataset above) to investigate for logistic regression classification? Apache Spark implementation is strongly recommended!

In [1]:
#Load libraries
import findspark
findspark.init()

import boto3
import io
from io import BytesIO
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import Row
import collections
from pyspark.sql.functions import col
import pyspark.sql.functions as fn
from pyspark.sql.types import *
import sys
import datetime

In [2]:
spark=SparkSession.builder \
                .master("local[4]") \
                .appName("string operations") \
                .config("spark.driver.memory","2g") \
                .config("spark.executor.memory","4g") \
                .config("spark.sql.shuffle.partitions", 150) \
                .getOrCreate()

In [3]:
sc=spark.sparkContext

# Read data

In [5]:
df=spark.read \
         .option("sep",",") \
         .option("header","True") \
         .option("inferSchema","True") \
         .csv("ahanda.csv")

In [6]:
df.limit(10).toPandas().head(5)

,_c0,ApplicationName,Domain,Url,MachineName,@timestamp,Exception,Client-IP,Logger,host,Client-IP1,Host-IP,date_day,date_hour
0,0,WWW.KANALD.COM.TR,/Form/Application,http://www.kanald.com.tr/Form/Application,KDWEBPROD05,2019-03-08 13:07:24.088,System.InvalidOperationException,None,KanalD.Framework.ExceptionFilter.KanaldExcepti...,192.168.112.247,None,None,2019-03-08,10
1,1,HURRIYET.COM.TR,/haberleri/kkk-tsk,http://www.hurriyet.com.tr/haberleri/kkk-tsk,HICMSWEB12,2019-03-08 13:07:24.065,System.Web.HttpException (0x80004005),"95.128.60.154, 192.168.113.177",ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,95.128.60.154,192.168.113.177,2019-03-08,10
2,2,HURRIYET.COM.TR,/bursa-buyuksehir-ettu-cupta-final-ariyor-4042...,http://www.hurriyet.com.tr/bursa-buyuksehir-et...,HICMSWEB07,2019-03-08 13:07:24.303,System.Net.WebException,"46.20.153.17, 192.168.113.178",Jumanji.Framework.Helper.CityHelper,192.168.112.247,46.20.153.17,192.168.113.178,2019-03-08,10
3,3,HURRIYET.COM.TR,/tunceli-haberleri/,http://www.hurriyet.com.tr/tunceli-haberleri/,HICMSWEB01,2019-03-08 13:07:15.961,System.Net.WebException,"46.20.153.22, 192.168.113.177",Jumanji.Framework.Helper.CityHelper,192.168.112.247,46.20.153.22,192.168.113.177,2019-03-08,10
4,4,HURRIYET.COM.TR,/tunceli-haberleri/,http://www.hurriyet.com.tr/tunceli-haberleri/,HICMSWEB01,2019-03-08 13:07:15.992,System.Net.WebException,"46.20.153.22, 192.168.113.177",Jumanji.Controllers.ControlController,192.168.112.247,46.20.153.22,192.168.113.177,2019-03-08,10


In [7]:
df2=df.repartition('Exception')

In [8]:
df2.rdd.getNumPartitions()

150

In [9]:
df2["ApplicationName"]

Column<b'ApplicationName'>

In [10]:
# take df2 as a List 
df2.head(2)[0]

Row(_c0=251, ApplicationName='KDQ.DOGANNET.TV', Domain="/api/search?q=M%C3%BCge%20ve%20G%C3%BCl%C5%9Fen'le%202.%20Sayfa%20Sinem%20Uma%C5%9F%20%C3%A7ileden%20%C3%A7%C4%B1kt%C4%B1!%20Canl%C4%B1%20yay%C4%B1nda%20birbirlerine%20girdiler!%20M%C3%BCge%20Da%C4%9F%C4%B1stanl%C4%B1%20G%C3%BCl%C5%9Fen%20Y%C3%BCksel%20Salt&p=/programlar/guncel/muge-ve-gulsenle-2-sayfa/&s=0&l=4&t=Article,PhotoGallery,Clip&o=OR&sf=StartDate&so=-1&f=Title,Tags", Url='http://kdq.dogannet.tv/api/search', MachineName='V-QUARK-4-86', @timestamp=datetime.datetime(2019, 3, 8, 13, 7, 23, 813000), Exception='Newtonsoft.Json.JsonReaderException', Client-IP=None, Logger='Quark.Models.Services.Contents.SearchService', host='192.168.112.247', Client-IP1=None, Host-IP=None, date_day=datetime.datetime(2019, 3, 8, 0, 0), date_hour=10)

In [11]:
# Column
type(df2["ApplicationName"])

pyspark.sql.column.Column

In [12]:
# Dataframe
type(df2.select("ApplicationName"))

pyspark.sql.dataframe.DataFrame

In [13]:
# List
type(df2.head(2)[0])

pyspark.sql.types.Row

# 1. Data Understanding
- utf-8 control
- title control
- seperator
- categoric & numeric variable understanding : (checking out outliers / removals / nulls etc.)
- printSchema() ile data herhangi bir kolon formata ihtiyac varmı?
- output değişkenin belirlenmesi

In [14]:
from functools import reduce

cols = ['_c0']
df2 = reduce(lambda x,y: x.drop(y),cols,df)
df2.limit(10).toPandas().head(5)

,ApplicationName,Domain,Url,MachineName,@timestamp,Exception,Client-IP,Logger,host,Client-IP1,Host-IP,date_day,date_hour
0,WWW.KANALD.COM.TR,/Form/Application,http://www.kanald.com.tr/Form/Application,KDWEBPROD05,2019-03-08 13:07:24.088,System.InvalidOperationException,None,KanalD.Framework.ExceptionFilter.KanaldExcepti...,192.168.112.247,None,None,2019-03-08,10
1,HURRIYET.COM.TR,/haberleri/kkk-tsk,http://www.hurriyet.com.tr/haberleri/kkk-tsk,HICMSWEB12,2019-03-08 13:07:24.065,System.Web.HttpException (0x80004005),"95.128.60.154, 192.168.113.177",ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,95.128.60.154,192.168.113.177,2019-03-08,10
2,HURRIYET.COM.TR,/bursa-buyuksehir-ettu-cupta-final-ariyor-4042...,http://www.hurriyet.com.tr/bursa-buyuksehir-et...,HICMSWEB07,2019-03-08 13:07:24.303,System.Net.WebException,"46.20.153.17, 192.168.113.178",Jumanji.Framework.Helper.CityHelper,192.168.112.247,46.20.153.17,192.168.113.178,2019-03-08,10
3,HURRIYET.COM.TR,/tunceli-haberleri/,http://www.hurriyet.com.tr/tunceli-haberleri/,HICMSWEB01,2019-03-08 13:07:15.961,System.Net.WebException,"46.20.153.22, 192.168.113.177",Jumanji.Framework.Helper.CityHelper,192.168.112.247,46.20.153.22,192.168.113.177,2019-03-08,10
4,HURRIYET.COM.TR,/tunceli-haberleri/,http://www.hurriyet.com.tr/tunceli-haberleri/,HICMSWEB01,2019-03-08 13:07:15.992,System.Net.WebException,"46.20.153.22, 192.168.113.177",Jumanji.Controllers.ControlController,192.168.112.247,46.20.153.22,192.168.113.177,2019-03-08,10


In [15]:
df2.count()

1000

In [16]:
df2.printSchema()

root
 |-- ApplicationName: string (nullable = true)
 |-- Domain: string (nullable = true)
 |-- Url: string (nullable = true)
 |-- MachineName: string (nullable = true)
 |-- @timestamp: timestamp (nullable = true)
 |-- Exception: string (nullable = true)
 |-- Client-IP: string (nullable = true)
 |-- Logger: string (nullable = true)
 |-- host: string (nullable = true)
 |-- Client-IP1: string (nullable = true)
 |-- Host-IP: string (nullable = true)
 |-- date_day: timestamp (nullable = true)
 |-- date_hour: integer (nullable = true)



# Categoric Variables Check
- farklı categori degerler araya girmiş mi?
- null degerler varmı? varsa bunları dolduruyoruz.
- categoriler hakkında bilgi edinme

In [10]:
df.createOrReplaceTempView("table1")

### a. ApplicationName

In [11]:
spark.sql("SELECT ApplicationName,count(*) as count FROM table1 GROUP BY ApplicationName ORDER BY count DESC").toPandas().head(20)

,ApplicationName,count
0,HURRIYET.COM.TR,596
1,Election.Framework,273
2,MOBILAPI.HURRIYET.COM.TR,46
3,WWW.KANALD.COM.TR,30
4,UYELIKYONETIM.HURRIYET.COM.TR,16
5,CMSAPIMOBIL.HURRIYET.COM.TR,14
6,CMS.HURRIYET.COM.TR,8
7,None,6
8,PILOT.HURRIYET.COM.TR,3
9,SansOyunlari.API,3


**Findings:** count en az SpaceJam.Consumer	 o halde bunu cıkarabilirz.

### b. Domain

In [12]:
spark.sql("SELECT Domain,count(*) as countt FROM table1 GROUP BY Domain ORDER BY countt DESC").toPandas().head(20)

,Domain,countt
0,None,310
1,/Form/Application,13
2,/api/NewsPhotoGalleries/Active()?$orderby=Star...,9
3,/amp/gundem/sumeyye-erdoganin-gelin-buketini-o...,4
4,/,3
5,/giresun-haberleri/,3
6,/api/Columns/Active()?$orderby=StartDate%20des...,3
7,/tunceli-haberleri/,2
8,/api/Articles/Active()?$orderby=StartDate%20de...,2
9,/haberleri/mcks?a=Melike%20Karakartal&r=1,1


### c. Url

In [13]:
spark.sql("SELECT Url,count(*) as countt FROM table1 GROUP BY Url ORDER BY countt DESC").toPandas().head(20)

,Url,countt
0,None,307
1,http://mobilapi.hurriyet.com.tr/api/authors/of...,30
2,http://www.kanald.com.tr/Form/Application,13
3,http://cmsapimobil.hurriyet.com.tr/api/NewsPho...,9
4,http://mobilapi.hurriyet.com.tr/api/authors/ge...,6
5,http://mobilapi.hurriyet.com.tr/api/sporarena/...,5
6,http://www.hurriyet.com.tr/amp/gundem/sumeyye-...,4
7,http://sansoyunlari.hurriyet.com.tr/api/change...,3
8,http://kdq.dogannet.tv/api/search,3
9,http://www.hurriyet.com.tr/giresun-haberleri/,3


### d. MachineName

In [14]:
spark.sql("SELECT MachineName,count(*) as countt FROM table1 GROUP BY MachineName ORDER BY countt DESC").toPandas().head(30)

,MachineName,countt
0,SECIM_2018_WEB-PRODUCTION_3_DOCKERLNXCLS105,91
1,SECIM_2018_WEB-PRODUCTION_2_DOCKERLNXCLS114,71
2,HICMSWEB04,64
3,HICMSWEB03,61
4,HICMSWEB08,56
5,HICMSWEB01,53
6,HICMSWEB13,48
7,HICMSWEB05,47
8,HICMSWEB10,47
9,HICMSWEB12,45


### e.Exception

In [15]:
spark.sql("SELECT Exception,count(*) as countt FROM table1 GROUP BY Exception ORDER BY countt DESC").toPandas().head(30)

,Exception,countt
0,System.Web.HttpException (0x80004005),391
1,None,330
2,System.Net.WebException,221
3,Microsoft.Data.OData.ODataException,14
4,System.FormatException,14
5,System.InvalidOperationException,13
6,System.ArgumentException,10
7,Newtonsoft.Json.JsonReaderException,3
8,System.UriFormatException,2
9,System.Text.RegularExpressions.RegexMatchTimeo...,2


### f. Logger

In [16]:
spark.sql("SELECT Logger,count(*) as countt FROM table1 GROUP BY Logger ORDER BY countt DESC").toPandas().head(30)

,Logger,countt
0,ASP._Page_Views_Control_ElasticSearch_cshtml,374
1,StackExchange.Profiling.Internal.ProfilingView...,273
2,Jumanji.Framework.Helper.CityHelper,201
3,HurriyetMobileApi.Controllers.AuthorsController,36
4,KanalD.Framework.ExceptionFilter.KanaldExcepti...,30
5,Quark.NLogExceptionLogger,14
6,Hangfire.AutomaticRetryAttribute,10
7,Jumanji.Controllers.ControlController,9
8,HurriyetMobileApi.Helper.CmsHelper,8
9,None,6


### g. Client-IP

In [17]:
df2.groupBy(["Client-IP"]).agg(fn.count(col("@timestamp")).alias("count_exception")).orderBy("count_exception",ascending=False).toPandas().head(20)

,Client-IP,count_exception
0,None,343
1,"95.128.60.154, 192.168.113.177",75
2,"95.128.60.154, 192.168.113.178",69
3,"46.20.153.2, 192.168.113.178",53
4,"46.20.153.4, 192.168.113.177",51
5,"46.20.153.2, 192.168.113.177",49
6,"46.20.153.4, 192.168.113.178",48
7,"46.20.153.14, 192.168.113.177",22
8,"46.20.153.22, 192.168.113.178",19
9,"46.20.153.17, 192.168.113.178",16


### h. host

In [18]:
df2.groupBy(["host"]).agg(fn.count(col("@timestamp")).alias("count_exception")).orderBy("count_exception",ascending=False).toPandas().head(20)

,host,count_exception
0,192.168.112.247,1000


### i. Host-IP

In [19]:
df2.groupBy(["Host-IP"]).agg(fn.count(col("@timestamp")).alias("count_exception")).orderBy("count_exception",ascending=False).toPandas().head(20)

,Host-IP,count_exception
0,None,390
1,192.168.113.177,315
2,192.168.113.178,281
3,192.168.113.71,8
4,192.168.113.70,6


# 2.  Data Preparation 
Data Understanding sonucu plan:
- Date formatting
- Removing unnecessary columns
- Tüm sütunlarda bosluk kontrolü yap
- null degerlerle ilgili karar verilecek
- timestamp ay/gün bazında yeni kolonlara alınacak
- cok az tekrarlanan categoric verilerin datasetinden cıkartılması

### a. Date Format 

In [20]:
current_format="yyyy-MM-dd HH:mm:ss.SSSZ"

In [21]:
from pyspark.sql import functions as F

df3=df2.withColumn("year",F.year(F.col("@timestamp"))) \
       .withColumn("month",F.month(F.col("@timestamp"))) \
       .withColumn("day",F.dayofmonth(F.col("@timestamp"))) \
       .withColumn("hour",F.hour(F.col("@timestamp"))) \
       .withColumn("new_date",F.to_date(F.col("date_day"),current_format)) 
       

In [22]:
df3.toPandas().head(20)

,ApplicationName,Domain,Url,MachineName,@timestamp,Exception,Client-IP,Logger,host,Client-IP1,Host-IP,date_day,date_hour,year,month,day,hour,new_date
0,WWW.KANALD.COM.TR,/Form/Application,http://www.kanald.com.tr/Form/Application,KDWEBPROD05,2019-03-08 13:07:24.088,System.InvalidOperationException,None,KanalD.Framework.ExceptionFilter.KanaldExcepti...,192.168.112.247,None,None,2019-03-08,10,2019,3,8,13,2019-03-08
1,HURRIYET.COM.TR,/haberleri/kkk-tsk,http://www.hurriyet.com.tr/haberleri/kkk-tsk,HICMSWEB12,2019-03-08 13:07:24.065,System.Web.HttpException (0x80004005),"95.128.60.154, 192.168.113.177",ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,95.128.60.154,192.168.113.177,2019-03-08,10,2019,3,8,13,2019-03-08
2,HURRIYET.COM.TR,/bursa-buyuksehir-ettu-cupta-final-ariyor-4042...,http://www.hurriyet.com.tr/bursa-buyuksehir-et...,HICMSWEB07,2019-03-08 13:07:24.303,System.Net.WebException,"46.20.153.17, 192.168.113.178",Jumanji.Framework.Helper.CityHelper,192.168.112.247,46.20.153.17,192.168.113.178,2019-03-08,10,2019,3,8,13,2019-03-08
3,HURRIYET.COM.TR,/tunceli-haberleri/,http://www.hurriyet.com.tr/tunceli-haberleri/,HICMSWEB01,2019-03-08 13:07:15.961,System.Net.WebException,"46.20.153.22, 192.168.113.177",Jumanji.Framework.Helper.CityHelper,192.168.112.247,46.20.153.22,192.168.113.177,2019-03-08,10,2019,3,8,13,2019-03-08
4,HURRIYET.COM.TR,/tunceli-haberleri/,http://www.hurriyet.com.tr/tunceli-haberleri/,HICMSWEB01,2019-03-08 13:07:15.992,System.Net.WebException,"46.20.153.22, 192.168.113.177",Jumanji.Controllers.ControlController,192.168.112.247,46.20.153.22,192.168.113.177,2019-03-08,10,2019,3,8,13,2019-03-08
5,HURRIYET.COM.TR,/haberleri/shahid-kapoor,http://www.hurriyet.com.tr/haberleri/shahid-ka...,HICMSWEB04,2019-03-08 13:07:16.030,System.Web.HttpException (0x80004005),"95.128.60.154, 192.168.113.178",ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,95.128.60.154,192.168.113.178,2019-03-08,10,2019,3,8,13,2019-03-08
6,HURRIYET.COM.TR,/haberleri/mcks?a=Melike%20Karakartal&r=1,http://www.hurriyet.com.tr/haberleri/mcks,HICMSWEB10,2019-03-08 13:07:16.154,System.Web.HttpException (0x80004005),"46.20.153.2, 192.168.113.177",ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,46.20.153.2,192.168.113.177,2019-03-08,10,2019,3,8,13,2019-03-08
7,HURRIYET.COM.TR,/haberleri/bjk,http://www.hurriyet.com.tr/haberleri/bjk,HICMSWEB03,2019-03-08 13:07:16.073,System.Web.HttpException (0x80004005),"95.128.60.154, 192.168.113.177",ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,95.128.60.154,192.168.113.177,2019-03-08,10,2019,3,8,13,2019-03-08
8,HURRIYET.COM.TR,/haberleri/ovacik-belediyesi,http://www.hurriyet.com.tr/haberleri/ovacik-be...,HICMSWEB04,2019-03-08 13:07:16.139,System.Web.HttpException (0x80004005),"46.20.153.4, 192.168.113.178",ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,46.20.153.4,192.168.113.178,2019-03-08,10,2019,3,8,13,2019-03-08
9,HURRIYET.COM.TR,/haberleri/hastaneye-kaldirildi,http://www.hurriyet.com.tr/haberleri/hastaneye...,HICMSWEB13,2019-03-08 13:07:16.160,System.Web.HttpException (0x80004005),"95.128.60.154, 192.168.113.178",ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,95.128.60.154,192.168.113.178,2019-03-08,10,2019,3,8,13,2019-03-08


In [23]:
df3.printSchema()

root
 |-- ApplicationName: string (nullable = true)
 |-- Domain: string (nullable = true)
 |-- Url: string (nullable = true)
 |-- MachineName: string (nullable = true)
 |-- @timestamp: timestamp (nullable = true)
 |-- Exception: string (nullable = true)
 |-- Client-IP: string (nullable = true)
 |-- Logger: string (nullable = true)
 |-- host: string (nullable = true)
 |-- Client-IP1: string (nullable = true)
 |-- Host-IP: string (nullable = true)
 |-- date_day: timestamp (nullable = true)
 |-- date_hour: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- new_date: date (nullable = true)



### b. Remove unnecessary columns

In [24]:
from functools import reduce

cols = ['date_day','date_hour']
df4 = reduce(lambda x,y: x.drop(y),cols,df3)
df4.limit(10).toPandas().head(5)

,ApplicationName,Domain,Url,MachineName,@timestamp,Exception,Client-IP,Logger,host,Client-IP1,Host-IP,year,month,day,hour,new_date
0,WWW.KANALD.COM.TR,/Form/Application,http://www.kanald.com.tr/Form/Application,KDWEBPROD05,2019-03-08 13:07:24.088,System.InvalidOperationException,None,KanalD.Framework.ExceptionFilter.KanaldExcepti...,192.168.112.247,None,None,2019,3,8,13,2019-03-08
1,HURRIYET.COM.TR,/haberleri/kkk-tsk,http://www.hurriyet.com.tr/haberleri/kkk-tsk,HICMSWEB12,2019-03-08 13:07:24.065,System.Web.HttpException (0x80004005),"95.128.60.154, 192.168.113.177",ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,95.128.60.154,192.168.113.177,2019,3,8,13,2019-03-08
2,HURRIYET.COM.TR,/bursa-buyuksehir-ettu-cupta-final-ariyor-4042...,http://www.hurriyet.com.tr/bursa-buyuksehir-et...,HICMSWEB07,2019-03-08 13:07:24.303,System.Net.WebException,"46.20.153.17, 192.168.113.178",Jumanji.Framework.Helper.CityHelper,192.168.112.247,46.20.153.17,192.168.113.178,2019,3,8,13,2019-03-08
3,HURRIYET.COM.TR,/tunceli-haberleri/,http://www.hurriyet.com.tr/tunceli-haberleri/,HICMSWEB01,2019-03-08 13:07:15.961,System.Net.WebException,"46.20.153.22, 192.168.113.177",Jumanji.Framework.Helper.CityHelper,192.168.112.247,46.20.153.22,192.168.113.177,2019,3,8,13,2019-03-08
4,HURRIYET.COM.TR,/tunceli-haberleri/,http://www.hurriyet.com.tr/tunceli-haberleri/,HICMSWEB01,2019-03-08 13:07:15.992,System.Net.WebException,"46.20.153.22, 192.168.113.177",Jumanji.Controllers.ControlController,192.168.112.247,46.20.153.22,192.168.113.177,2019,3,8,13,2019-03-08


### c. Null kontrolü

In [25]:
sayac_for_null=1
for sutun in df4.columns:
    if(df4.filter(F.col(sutun).isNull()).count() > 0):
        print(sayac_for_null,sutun,"---> ",df4.filter(F.col(sutun).isNull()).count()," adet null var")
    else:
        print(sayac_for_null,sutun)
    sayac_for_null +=1

1 ApplicationName --->  6  adet null var
2 Domain --->  310  adet null var
3 Url --->  307  adet null var
4 MachineName --->  6  adet null var
5 @timestamp
6 Exception --->  330  adet null var
7 Client-IP --->  343  adet null var
8 Logger --->  6  adet null var
9 host
10 Client-IP1 --->  343  adet null var
11 Host-IP --->  390  adet null var
12 year
13 month
14 day
15 hour
16 new_date


### d. Peki Null karakteri rastgelemi gelmiş yoksa belli output sonuclarında mı gelmiş ?

In [26]:
df4.select("ApplicationName","MachineName","Logger","Domain","Url","Exception","Client-IP1","Host-IP") \
.filter(
                F.col("ApplicationName").isNull() | \
                F.col("MachineName").isNull() | \
                F.col("Logger").isNull() | \
                F.col("Domain").isNull() | \
                F.col("Url").isNull() | \
                F.col("Exception").isNull() | \
                F.col("Client-IP1").isNull() | \
                F.col("Host-IP").isNull() 
).groupBy("ApplicationName","MachineName","Logger","Domain","Url","Exception","Client-IP1","Host-IP").count() \
 .orderBy(F.col("count").desc()) \
 .toPandas().head(150)

,ApplicationName,MachineName,Logger,Domain,Url,Exception,Client-IP1,Host-IP,count
0,Election.Framework,SECIM_2018_WEB-PRODUCTION_3_DOCKERLNXCLS105,StackExchange.Profiling.Internal.ProfilingView...,None,None,None,None,None,91
1,Election.Framework,SECIM_2018_WEB-PRODUCTION_2_DOCKERLNXCLS114,StackExchange.Profiling.Internal.ProfilingView...,None,None,None,None,None,71
2,Election.Framework,SECIM_2018_WEB-PRODUCTION_4_DOCKERLNXCLS112,StackExchange.Profiling.Internal.ProfilingView...,None,None,None,None,None,41
3,Election.Framework,SECIM_2018_WEB-PRODUCTION_5_DOCKERLNXCLS109,StackExchange.Profiling.Internal.ProfilingView...,None,None,None,None,None,35
4,Election.Framework,SECIM_2018_WEB-PRODUCTION_1_DOCKERLNXCLS108,StackExchange.Profiling.Internal.ProfilingView...,None,None,None,None,None,35
5,UYELIKYONETIM.HURRIYET.COM.TR,UYELIKYONETIM-2-DOCKERLNXCLS112,Hangfire.AutomaticRetryAttribute,None,None,System.FormatException,None,None,10
6,None,None,None,None,None,None,None,None,6
7,WWW.KANALD.COM.TR,KDWEBPROD08,KanalD.Framework.ExceptionFilter.KanaldExcepti...,/Form/Application,http://www.kanald.com.tr/Form/Application,System.InvalidOperationException,None,None,4
8,UYELIKYONETIM.HURRIYET.COM.TR,UYELIKYONETIM-2-DOCKERLNXCLS112,UY.Service.JobService,None,None,System.FormatException,None,None,4
9,CMS.HURRIYET.COM.TR,HICMSAPP02,HurriyetQuarkServicePlugin.Helpers.VarnishHelper,None,None,None,None,None,3


### e. Null olanları datasetinden cıkaralım.

In [27]:
df5=df4 \
        .filter(~(
                F.col("ApplicationName").isNull() | \
                F.col("MachineName").isNull() | \
                F.col("Logger").isNull() | \
                F.col("Domain").isNull() | \
                F.col("Url").isNull() | \
                F.col("Exception").isNull() | \
                F.col("Client-IP1").isNull() | \
                F.col("Host-IP").isNull() 
))

print(df4.count())
print(df5.count())

1000
610


**Elimizdeki datasetimizin toplam observation sayısı: 610**

In [28]:
df5.toPandas().head(10)

,ApplicationName,Domain,Url,MachineName,@timestamp,Exception,Client-IP,Logger,host,Client-IP1,Host-IP,year,month,day,hour,new_date
0,HURRIYET.COM.TR,/haberleri/kkk-tsk,http://www.hurriyet.com.tr/haberleri/kkk-tsk,HICMSWEB12,2019-03-08 13:07:24.065,System.Web.HttpException (0x80004005),"95.128.60.154, 192.168.113.177",ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,95.128.60.154,192.168.113.177,2019,3,8,13,2019-03-08
1,HURRIYET.COM.TR,/bursa-buyuksehir-ettu-cupta-final-ariyor-4042...,http://www.hurriyet.com.tr/bursa-buyuksehir-et...,HICMSWEB07,2019-03-08 13:07:24.303,System.Net.WebException,"46.20.153.17, 192.168.113.178",Jumanji.Framework.Helper.CityHelper,192.168.112.247,46.20.153.17,192.168.113.178,2019,3,8,13,2019-03-08
2,HURRIYET.COM.TR,/tunceli-haberleri/,http://www.hurriyet.com.tr/tunceli-haberleri/,HICMSWEB01,2019-03-08 13:07:15.961,System.Net.WebException,"46.20.153.22, 192.168.113.177",Jumanji.Framework.Helper.CityHelper,192.168.112.247,46.20.153.22,192.168.113.177,2019,3,8,13,2019-03-08
3,HURRIYET.COM.TR,/tunceli-haberleri/,http://www.hurriyet.com.tr/tunceli-haberleri/,HICMSWEB01,2019-03-08 13:07:15.992,System.Net.WebException,"46.20.153.22, 192.168.113.177",Jumanji.Controllers.ControlController,192.168.112.247,46.20.153.22,192.168.113.177,2019,3,8,13,2019-03-08
4,HURRIYET.COM.TR,/haberleri/shahid-kapoor,http://www.hurriyet.com.tr/haberleri/shahid-ka...,HICMSWEB04,2019-03-08 13:07:16.030,System.Web.HttpException (0x80004005),"95.128.60.154, 192.168.113.178",ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,95.128.60.154,192.168.113.178,2019,3,8,13,2019-03-08
5,HURRIYET.COM.TR,/haberleri/mcks?a=Melike%20Karakartal&r=1,http://www.hurriyet.com.tr/haberleri/mcks,HICMSWEB10,2019-03-08 13:07:16.154,System.Web.HttpException (0x80004005),"46.20.153.2, 192.168.113.177",ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,46.20.153.2,192.168.113.177,2019,3,8,13,2019-03-08
6,HURRIYET.COM.TR,/haberleri/bjk,http://www.hurriyet.com.tr/haberleri/bjk,HICMSWEB03,2019-03-08 13:07:16.073,System.Web.HttpException (0x80004005),"95.128.60.154, 192.168.113.177",ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,95.128.60.154,192.168.113.177,2019,3,8,13,2019-03-08
7,HURRIYET.COM.TR,/haberleri/ovacik-belediyesi,http://www.hurriyet.com.tr/haberleri/ovacik-be...,HICMSWEB04,2019-03-08 13:07:16.139,System.Web.HttpException (0x80004005),"46.20.153.4, 192.168.113.178",ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,46.20.153.4,192.168.113.178,2019,3,8,13,2019-03-08
8,HURRIYET.COM.TR,/haberleri/hastaneye-kaldirildi,http://www.hurriyet.com.tr/haberleri/hastaneye...,HICMSWEB13,2019-03-08 13:07:16.160,System.Web.HttpException (0x80004005),"95.128.60.154, 192.168.113.178",ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,95.128.60.154,192.168.113.178,2019,3,8,13,2019-03-08
9,HURRIYET.COM.TR,/anaokulu-ogrencilerinden-vaski-ye-ziyaret-371...,http://www.hurriyet.com.tr/anaokulu-ogrenciler...,HICMSWEB06,2019-03-08 13:07:16.138,System.Net.WebException,"46.20.153.20, 192.168.113.177",Jumanji.Framework.Helper.CityHelper,192.168.112.247,46.20.153.20,192.168.113.177,2019,3,8,13,2019-03-08


### f. Variable'ların sıralanması

In [29]:
df5.columns

['ApplicationName',
 'Domain',
 'Url',
 'MachineName',
 '@timestamp',
 'Exception',
 'Client-IP',
 'Logger',
 'host',
 'Client-IP1',
 'Host-IP',
 'year',
 'month',
 'day',
 'hour',
 'new_date']

In [30]:
feature_order=["ApplicationName", "Domain", "Url", "MachineName", "Logger", "host","Client-IP1", "Host-IP", "@timestamp", "new_date", "year", "month", "day", "hour", "Exception"]

In [31]:
df6=df5.select(feature_order)
df6.toPandas().head(10)

,ApplicationName,Domain,Url,MachineName,Logger,host,Client-IP1,Host-IP,@timestamp,new_date,year,month,day,hour,Exception
0,HURRIYET.COM.TR,/haberleri/kkk-tsk,http://www.hurriyet.com.tr/haberleri/kkk-tsk,HICMSWEB12,ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,95.128.60.154,192.168.113.177,2019-03-08 13:07:24.065,2019-03-08,2019,3,8,13,System.Web.HttpException (0x80004005)
1,HURRIYET.COM.TR,/bursa-buyuksehir-ettu-cupta-final-ariyor-4042...,http://www.hurriyet.com.tr/bursa-buyuksehir-et...,HICMSWEB07,Jumanji.Framework.Helper.CityHelper,192.168.112.247,46.20.153.17,192.168.113.178,2019-03-08 13:07:24.303,2019-03-08,2019,3,8,13,System.Net.WebException
2,HURRIYET.COM.TR,/tunceli-haberleri/,http://www.hurriyet.com.tr/tunceli-haberleri/,HICMSWEB01,Jumanji.Framework.Helper.CityHelper,192.168.112.247,46.20.153.22,192.168.113.177,2019-03-08 13:07:15.961,2019-03-08,2019,3,8,13,System.Net.WebException
3,HURRIYET.COM.TR,/tunceli-haberleri/,http://www.hurriyet.com.tr/tunceli-haberleri/,HICMSWEB01,Jumanji.Controllers.ControlController,192.168.112.247,46.20.153.22,192.168.113.177,2019-03-08 13:07:15.992,2019-03-08,2019,3,8,13,System.Net.WebException
4,HURRIYET.COM.TR,/haberleri/shahid-kapoor,http://www.hurriyet.com.tr/haberleri/shahid-ka...,HICMSWEB04,ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,95.128.60.154,192.168.113.178,2019-03-08 13:07:16.030,2019-03-08,2019,3,8,13,System.Web.HttpException (0x80004005)
5,HURRIYET.COM.TR,/haberleri/mcks?a=Melike%20Karakartal&r=1,http://www.hurriyet.com.tr/haberleri/mcks,HICMSWEB10,ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,46.20.153.2,192.168.113.177,2019-03-08 13:07:16.154,2019-03-08,2019,3,8,13,System.Web.HttpException (0x80004005)
6,HURRIYET.COM.TR,/haberleri/bjk,http://www.hurriyet.com.tr/haberleri/bjk,HICMSWEB03,ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,95.128.60.154,192.168.113.177,2019-03-08 13:07:16.073,2019-03-08,2019,3,8,13,System.Web.HttpException (0x80004005)
7,HURRIYET.COM.TR,/haberleri/ovacik-belediyesi,http://www.hurriyet.com.tr/haberleri/ovacik-be...,HICMSWEB04,ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,46.20.153.4,192.168.113.178,2019-03-08 13:07:16.139,2019-03-08,2019,3,8,13,System.Web.HttpException (0x80004005)
8,HURRIYET.COM.TR,/haberleri/hastaneye-kaldirildi,http://www.hurriyet.com.tr/haberleri/hastaneye...,HICMSWEB13,ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,95.128.60.154,192.168.113.178,2019-03-08 13:07:16.160,2019-03-08,2019,3,8,13,System.Web.HttpException (0x80004005)
9,HURRIYET.COM.TR,/anaokulu-ogrencilerinden-vaski-ye-ziyaret-371...,http://www.hurriyet.com.tr/anaokulu-ogrenciler...,HICMSWEB06,Jumanji.Framework.Helper.CityHelper,192.168.112.247,46.20.153.20,192.168.113.177,2019-03-08 13:07:16.138,2019-03-08,2019,3,8,13,System.Net.WebException


### g. Dataframe'i diske yazma

In [31]:
df6 \
      .coalesce(1) \
      .write \
      .mode("overwrite") \
      .option("sep",",") \
      .option("header","true") \
      .csv("preprocessed_log_2019-03-08_reduced")

# 3. Vectorization

In [4]:
df=spark.read \
         .option("sep",",") \
         .option("header","True") \
         .option("inferSchema","True") \
         .csv("preprocessed_log_2019-03-08_reduced")

In [5]:
from pyspark.ml.feature import StringIndexer,OneHotEncoder,OneHotEncoderEstimator,VectorAssembler,StandardScaler

In [6]:
df.toPandas().head()

,ApplicationName,Domain,Url,MachineName,Logger,host,Client-IP1,Host-IP,@timestamp,new_date,year,month,day,hour,Exception
0,HURRIYET.COM.TR,/haberleri/kkk-tsk,http://www.hurriyet.com.tr/haberleri/kkk-tsk,HICMSWEB12,ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,95.128.60.154,192.168.113.177,2019-03-08 13:07:24.065,2019-03-08,2019,3,8,13,System.Web.HttpException (0x80004005)
1,HURRIYET.COM.TR,/bursa-buyuksehir-ettu-cupta-final-ariyor-4042...,http://www.hurriyet.com.tr/bursa-buyuksehir-et...,HICMSWEB07,Jumanji.Framework.Helper.CityHelper,192.168.112.247,46.20.153.17,192.168.113.178,2019-03-08 13:07:24.303,2019-03-08,2019,3,8,13,System.Net.WebException
2,HURRIYET.COM.TR,/tunceli-haberleri/,http://www.hurriyet.com.tr/tunceli-haberleri/,HICMSWEB01,Jumanji.Framework.Helper.CityHelper,192.168.112.247,46.20.153.22,192.168.113.177,2019-03-08 13:07:15.961,2019-03-08,2019,3,8,13,System.Net.WebException
3,HURRIYET.COM.TR,/tunceli-haberleri/,http://www.hurriyet.com.tr/tunceli-haberleri/,HICMSWEB01,Jumanji.Controllers.ControlController,192.168.112.247,46.20.153.22,192.168.113.177,2019-03-08 13:07:15.992,2019-03-08,2019,3,8,13,System.Net.WebException
4,HURRIYET.COM.TR,/haberleri/shahid-kapoor,http://www.hurriyet.com.tr/haberleri/shahid-ka...,HICMSWEB04,ASP._Page_Views_Control_ElasticSearch_cshtml,192.168.112.247,95.128.60.154,192.168.113.178,2019-03-08 13:07:16.030,2019-03-08,2019,3,8,13,System.Web.HttpException (0x80004005)


In [7]:
train_df, test_df = df.randomSplit([0.8, 0.2], seed=142)

In [8]:
app_indexer = StringIndexer() \
.setInputCol("ApplicationName") \
.setOutputCol("ApplicationName_index") \
.setHandleInvalid("skip")

machine_indexer = StringIndexer() \
.setInputCol("MachineName") \
.setOutputCol("MachineName_index") \
.setHandleInvalid("skip")

# host_indexer = StringIndexer() \
# .setInputCol("host") \
# .setOutputCol("host_index") \
# .setHandleInvalid("skip")

clientIP_indexer = StringIndexer() \
.setInputCol("Client-IP1") \
.setOutputCol("Client-IP1_index") \
.setHandleInvalid("skip")

hostIP_indexer = StringIndexer() \
.setInputCol("Host-IP") \
.setOutputCol("Host-IP_index") \
.setHandleInvalid("skip")


In [9]:
encoder = OneHotEncoderEstimator() \
.setInputCols(["ApplicationName_index","MachineName_index","Client-IP1_index","Host-IP_index"]) \
.setOutputCols(["app_encoded","machine_encoded","clientIP_encoded","hostIP_encoded"])

In [10]:
assembler = VectorAssembler() \
.setInputCols(["app_encoded","machine_encoded","clientIP_encoded","hostIP_encoded","year","month","day","hour"]) \
.setOutputCol("vectorized_features")

In [11]:
label_indexer = StringIndexer() \
.setInputCol("Exception") \
.setOutputCol("label") \
.setHandleInvalid("skip")


In [12]:
# Label-Exception value pairs
label_model=label_indexer.fit(df2)
df_label_model=label_model.transform(df2)

df_label_model.toPandas().head(10)
df_excep_count=df_label_model.groupBy(F.col("label"),F.col("Exception")) \
            .agg(F.count(F.col("label"))) \
            .select(F.col("label"),F.col("Exception"),F.col("count(label)").alias("label_count")) \
            .sort(F.desc("label_count")).toPandas().head(100)

NameError: name 'df2' is not defined

In [ ]:
scaler = StandardScaler() \
.setInputCol("vectorized_features") \
.setOutputCol("features")

# 4 Modelling

In [42]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

lr_object = LogisticRegression(maxIter=100) \
.setFeaturesCol("features") \
.setLabelCol("label") \
.setPredictionCol("prediction")

In [43]:
pipeline_object = Pipeline() \
.setStages([app_indexer,machine_indexer,clientIP_indexer,hostIP_indexer,encoder,assembler,label_indexer,scaler,lr_object])

In [71]:
pipeline_model = pipeline_object.fit(train_df)
print(pipeline_model.stages[:])


[StringIndexer_4f5251f50af0, StringIndexer_33ba4fa20d66, StringIndexer_f8a05743934d, StringIndexer_2f6bcb76e9fa, OneHotEncoderEstimator_7e2998fe2410, VectorAssembler_dc2cb6588d23, StringIndexer_bc20c33ae124, StandardScaler_cf0d8c336cce, LogisticRegressionModel: uid = LogisticRegression_fc1de8e8f36f, numClasses = 5, numFeatures = 45]


In [105]:
pipeline_model.transform(test_df).select("label","prediction").toPandas().head(5)

,label,prediction
0,2.0,2.0
1,1.0,0.0
2,1.0,1.0
3,3.0,1.0
4,3.0,0.0


# 5. Model Evaluation

In [73]:
lr_model=pipeline_model.stages[8]

In [78]:
print("Coefficients: \n" + str(lr_model.coefficientMatrix))
print("Intercept: " + str(lr_model.interceptVector))

Coefficients: 
DenseMatrix([[ 2.08464911e-01,  2.36669826e-02,  1.09271377e-01,
               1.15694390e-01, -1.69711426e-02,  6.94327838e-02,
              -3.79306445e-02,  1.92501124e-02, -8.73535587e-02,
              -5.70156233e-02,  8.79908113e-02, -3.51180720e-02,
              -2.70830048e-02, -1.01999461e-01, -1.86309884e-01,
              -1.37983232e-01, -1.33724518e-01, -9.12712161e-02,
               7.01055914e-01,  1.76788557e-01,  1.07445057e-01,
              -7.08424906e-01, -2.08117742e-01, -3.33406716e-01,
              -1.56823318e-01, -5.48966594e-02,  5.00525919e-02,
              -1.86959697e-02, -2.26928629e-01, -3.01406802e-01,
              -1.79482055e-01, -2.85661372e-02,  9.99444460e-02,
              -4.23570214e-02,  8.52991260e-02, -8.44249194e-02,
               1.27816327e-02, -4.59903074e-02, -4.71932692e-02,
               1.06994419e-01, -2.27326974e-01,  0.00000000e+00,
               0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         

In [80]:
trainingSummary = lr_model.summary

In [81]:
# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

objectiveHistory:
0.8326335758478292
0.5788569459627413
0.5305867882321582
0.5279409803262664
0.5160786599516458
0.49792273580032365
0.4890833043314878
0.48227702631640607
0.47873270258856276
0.4711526710556379
0.46837371060503996


In [82]:
# for multiclass, we can inspect metrics on a per-label basis
print("False positive rate by label:")
for i, rate in enumerate(trainingSummary.falsePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print("True positive rate by label:")
for i, rate in enumerate(trainingSummary.truePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print("Precision by label:")
for i, prec in enumerate(trainingSummary.precisionByLabel):
    print("label %d: %s" % (i, prec))

print("Recall by label:")
for i, rec in enumerate(trainingSummary.recallByLabel):
    print("label %d: %s" % (i, rec))

print("F-measure by label:")
for i, f in enumerate(trainingSummary.fMeasureByLabel()):
    print("label %d: %s" % (i, f))

False positive rate by label:
label 0: 0.33157894736842103
label 1: 0.09935897435897435
label 2: 0.0
label 3: 0.002105263157894737
label 4: 0.0
True positive rate by label:
label 0: 0.9078498293515358
label 1: 0.6549707602339181
label 2: 1.0
label 3: 0.0
label 4: 0.0
Precision by label:
label 0: 0.8085106382978723
label 1: 0.7832167832167832
label 2: 1.0
label 3: 0.0
label 4: 0.0
Recall by label:
label 0: 0.9078498293515358
label 1: 0.6549707602339181
label 2: 1.0
label 3: 0.0
label 4: 0.0
F-measure by label:
label 0: 0.8553054662379421
label 1: 0.7133757961783439
label 2: 1.0
label 3: 0.0
label 4: 0.0


In [83]:
accuracy = trainingSummary.accuracy
falsePositiveRate = trainingSummary.weightedFalsePositiveRate
truePositiveRate = trainingSummary.weightedTruePositiveRate
fMeasure = trainingSummary.weightedFMeasure()
precision = trainingSummary.weightedPrecision
recall = trainingSummary.weightedRecall
print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

Accuracy: 0.8033126293995859
FPR: 0.23635581428487604
TPR: 0.8033126293995859
F-measure: 0.7921154508368817
Precision: 0.7884548384085849
Recall: 0.8033126293995859


# 6. Tunning

In [1]:
from lightning import Lightning
lgn = Lightning(local=True)

Lightning initialized
